# Accessing and managing groups
Groups are collaborative spaces in your GIS where your users share related content. Groups support metadata allowing you to customize them as per their theme. You can add users to a group with different group related privileges. Refer to the [topic on groups](https://doc.arcgis.com/en/arcgis-online/share-maps/groups.htm) to learn more about them.

In the `gis` module, groups are represented using instances of `Group` class. Similar to `Items`, `Users`, `Roles`, an instance of a resource manager class called `GroupManager` is used to create, search and provide you access to `Group` objects. As with other resource manager objects, you would not create an instance of `GroupManager` using its constructor, but access it from the `groups` property of the `GIS` object.

In this guide, we will observe:

<a id="searching-for-groups"></a>
## Searching for groups
You can search for groups by using the `search()` method of `GroupManager` object. The `search()` method accepts standard [ArcGIS REST API queries](http://resources.arcgis.com/en/help/arcgis-rest-api/#/Search_reference/02r3000000mn000000/). To illustrate this better, let us connect to ArcGIS Online anonymously and search for public groups that have the word 'water' in the title.

In [ ]:
from arcgis.gis import GIS
ago_gis = GIS()
urban_groups = ago_gis.groups.search('title:water', max_groups=15)
urban_groups

The `search()` method returns a list of objects of type `arcgis.gis.Group`. When using the Jupyter notebook environment, `Group` objects can be represented in rich HTML with their thumbnail and metadata information.

In [ ]:
urban_groups[1]

You can expand the query string to include multiple fields on a group such as `owner`, `description` etc. Let us look for groups created and owned by account `esri`.

In [ ]:
esri_owned_groups = ago_gis.groups.search(query='owner:esri and description:basemaps', max_groups=15)
esri_owned_groups

<a id="properties-of-a-group"></a>
## Properties of a group
You can query the metadata and related information about a group by querying the properties of its corresponding `Group` object:

In [ ]:
esri_group1 = esri_owned_groups[0]
esri_group1.access
esri_group1.__dict__

Let us print some more of the properties of this group

In [ ]:
import time
print(esri_group1.groupid, esri_group1.isFav, esri_group1.isInvitationOnly)
print(esri_group1.owner)
time.strftime("%m/%d/%Y, %H:%M:%S",time.localtime(esri_group1.created/1000))

Once you know the `id` of a group, you can access it using the `get()` method of the `GroupManager` object:

In [ ]:
ago_gis.groups.get(esri_group1.groupid)

<a id="creating-new-groups"></a>
## Creating new groups
You can create new groups by calling the `create()` method of the `GroupManager` object. This method is similar to other `create` methods of `ResourceManager` objects and returns you an instance of the `Group` object it created.

Let us create a new group for geocaching enthusiasts in our GIS. Replace the credentials below with that of your org. To learn more about profiles, [see here](https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/#Storing-your-credentialls-locally)

In [ ]:
# connect to GIS with profile
gis = GIS(profile="your_enterprise_profile")
myGroup = gis.groups.create(title='2022 Devsummit group',
                                    tags = 'group',
                                    description = 'Group to share your data',
                                    snippet = 'Share your data here',
                                    access = 'private',
                                    is_invitation_only = 'True',
                                    thumbnail = r'./img/wazeimg.png')
myGroup

<a id="sharing-content-to-groups"></a>
### Sharing content to groups
In an ArcGIS Org, an `Item` can have 4 privacy levels - `private`, `group`, `org`, `everyone`. When an `Item` is shared to a group, all members of a group get to view it. Similarly, when shared to an organization, all authenticated members of the org can view it. When shared to everyone, and if the org permits anonymous access, the item is public and accessible to anyone and does not have to be a logged in user.

First, let us add a CSV containing some points to the GIS.

In [ ]:
item_properties = {'title':'Earthquakes devsummit',
                  'tags':['Earthquakes'],
                  'snippet':'Earthquakes devsummit demo'}
fc_item = gis.content.add(item_properties, data='.\data\earthquakes.csv')

In [ ]:
fc_item.access

To share an `Item` to a group, call the `share()` method of the `Item` object and pass the `Group` object. You can also pass the name of the group or its ID.

In [ ]:
# let us share it to the group so other enthusiasts can enjoy this map
fc_item.share(groups=myGroup)

<a id="managing-your-groups"></a>
## Managing your groups

### Listing contents of the group
As the first step in managing your groups, let us view the items shared with the group by using the `content()` method

In [ ]:
myGroup.content()

### Adding users to the group
Now our group has sufficient details and content to make it useful for others. Let us add some users. You can either add users using the `add_users()` method and specify the list of usernames, or if you wish to leave joining to their choice, you can invite them to the group using the `invite_users()` method.

In [ ]:
# let us add 2 users to this group
myGroup.add_users(['student1', 'stud2'])

The method returns a dictionary with information on which users could not be added. As seen on the guide for [managing users](https://developers.arcgis.com/python/guide/accessing-and-managing-users/), users could have custom privileges  preventing them from joining groups. In such occasions, you would get the names of those users in this return list.

### Removing users from a group
You can remove users from a group by calling the `remove_users()` method and passing the list of usernames to be removed. The method returns you a dictionary containing those users who cannot be removed.

In [ ]:
# remove the one of the members
myGroup.remove_users(['student1'])

In [ ]:
# can you remove the owner of the group?
myGroup.remove_users(['arcgis_python_api'])

Thus, you cannot remove the owner or admin of the group until you reassign the ownership to another user.

### Listing users belonging to a group
You can list the users in a group by calling the `get_members()` method. The method returns a dictionary that not only gives you the member list, but also the owner, admin and users.

In [ ]:
myGroup.get_members()

### Updating a group
You can update any or all the fields of a group that you specified when creating it. Thus, the `update()` accepts the same parameters as `create()`. Let us close the group to members who can be added through invitation only. This prevents users from sending a joining request.

In [ ]:
myGroup.update(is_invitation_only=True)

In [ ]:
myGroup.isInvitationOnly

### Terminating a group
If a group no longer serves the purpose or if its requirements have changed, the owner and members have a few options. They can leave the group by calling the `leave()` method. When users leave a group, the content they shared with the group will automatically be unshared. The owner or org admin can delete the group by calling the `delete()` method. 

However if the contents and the group have to be preserved, the group's ownership can be transferred by calling the `reassign_to()` and specifying the new owner's username. 

<blockquote><b>Note:</b> Only the administrator of the org can reassign a group to another user.</blockquote>

let us reassign ownership to another user
```python
geocaching_group.reassign_to(target_owner = 'api_data_owner')
```

Finally, let us delete this group

In [ ]:
myGroup.delete()